In [12]:
import pandas as pd 
import numpy as np
import json 
import sklearn



In [58]:
df_train_small = pd.read_csv('./data_transformed/train_500_2.csv')


In [59]:
df_train_small

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,month,day,hour,average_lowest_gas_price,average_highest_gas_price
0,11,0,3,790.836,1,449,1414989,45,199.99,-4.8,2022,11,24,1,121.00,137.99
1,9,1,3,4.396,0,305,953684,37,211.31,19.0,2022,7,3,21,146.50,155.00
2,4,0,3,0.000,0,35,103626,16,98.25,8.7,2021,10,6,5,65.11,66.60
3,15,1,1,0.000,0,551,1745132,59,129.23,-4.0,2023,3,6,19,51.00,54.45
4,7,1,1,0.000,0,620,1961232,29,85.59,9.4,2023,5,14,2,38.60,41.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,2,0.000,0,253,786780,1,54.10,10.0,2022,5,12,23,98.00,104.12
499996,0,0,2,59.062,0,293,914348,1,199.04,12.5,2022,6,21,15,110.05,125.00
499997,4,1,0,902.756,1,268,833853,17,109.55,7.6,2022,5,27,14,102.00,121.95
499998,14,1,3,1935.450,1,63,188631,56,119.38,5.3,2021,11,3,7,59.00,87.90


In [71]:
# Supposons que df_train soit votre DataFrame

# Colonne pour laquelle vous souhaitez afficher les valeurs nulles
col_name = 'average_lowest_gas_price'

# Afficher les valeurs nulles dans la colonne spécifiée
null_values = df_train_small[col_name].isnull()

# Afficher les lignes où la valeur est nulle
print(df_train_small[null_values])


        county  is_business  product_type   target  is_consumption  \
410          7            1             1   21.889               0   
2900         4            0             3  128.483               0   
3372         3            0             3    6.037               1   
3565         7            0             1    8.261               1   
4599         9            1             3   79.820               1   
...        ...          ...           ...      ...             ...   
498592       0            1             1    2.133               0   
499271      10            0             1   11.351               1   
499356       2            0             3    0.000               0   
499394       8            0             1   11.895               1   
499431       9            1             3  180.818               0   

        data_block_id  row_id  prediction_unit_id  price_elec  temperature  \
410                 0    2254                  29      138.00         13.6   
290

In [68]:
# Supposons que df_train soit votre DataFrame

# Remplacer les NaN par "0" dans les colonnes spécifiées
df_train_small['average_highest_gas_price'].fillna(0, inplace=True)
df_train_small['average_lowest_gas_price'].fillna(0, inplace=True)

# Afficher les premières lignes du DataFrame df_train après le remplacement
print(df_train_small.head())


   county  is_business  product_type   target  is_consumption  data_block_id  \
0      11            0             3  790.836               1            449   
1       9            1             3    4.396               0            305   
2       4            0             3    0.000               0             35   
3      15            1             1    0.000               0            551   
4       7            1             1    0.000               0            620   

    row_id  prediction_unit_id  price_elec  temperature  year  month  day  \
0  1414989                  45      199.99         -4.8  2022     11   24   
1   953684                  37      211.31         19.0  2022      7    3   
2   103626                  16       98.25          8.7  2021     10    6   
3  1745132                  59      129.23         -4.0  2023      3    6   
4  1961232                  29       85.59          9.4  2023      5   14   

   hour  average_lowest_gas_price  average_highest_gas_p

C:\Users\Jawad\AppData\Local\Temp\ipykernel_21628\146887180.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_small['average_highest_gas_price'].fillna(0, inplace=True)
C:\Users\Jawad\AppData\Local\Temp\ipykernel_21628\146887180.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_small['average_lowest_gas_price'].fillna(0, inplace=True)


In [70]:
df_train_small.isna().sum()

county                       0
is_business                  0
product_type                 0
target                       0
is_consumption               0
data_block_id                0
row_id                       0
prediction_unit_id           0
price_elec                   0
temperature                  0
year                         0
month                        0
day                          0
hour                         0
average_lowest_gas_price     0
average_highest_gas_price    0
dtype: int64

In [72]:
# Drop all lines where target is null or nan
# A function 'dropna' exists also
df_train_small = df_train_small[df_train_small["target"].isna() == False]


# Train - Test 

In [73]:
from sklearn.model_selection import train_test_split

X = df_train_small.drop('target', axis=1)
y = df_train_small['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [74]:
X_train

,county,is_business,product_type,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,month,day,hour,average_lowest_gas_price,average_highest_gas_price
427040,11,1,1,0,27,80124,46,190.20,8.5,2021,9,28,8,49.51,54.76
349577,0,1,0,1,573,1812649,3,53.49,-1.8,2023,3,28,3,43.00,48.50
276127,0,1,2,0,632,2001790,61,55.08,12.1,2023,5,26,17,30.00,35.00
23230,7,0,3,0,396,1241534,27,357.80,9.7,2022,10,2,6,185.00,201.60
391151,9,0,3,0,231,712630,35,30.42,5.7,2022,4,20,1,96.76,119.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259243,13,0,3,0,403,1264432,51,372.19,9.7,2022,10,9,6,130.00,144.29
365935,11,1,1,0,594,1879802,46,35.36,4.6,2023,4,18,12,47.21,51.49
131963,4,0,3,0,570,1804046,16,49.62,3.9,2023,3,25,10,43.10,46.99
146904,9,0,1,1,469,1480577,34,338.54,14.0,2022,12,14,6,131.01,157.00


In [75]:
y_train.isna().sum()

0

In [76]:
# Entrainement 
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)

DummyRegressor()

In [77]:
#Test 
from sklearn.dummy import DummyRegressor

dummy_regr_test = DummyRegressor(strategy="mean")
dummy_regr_test.fit(X_test, y_test)

DummyRegressor()

In [78]:
# Entrainement 
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)
y_train_hat = reg.predict(X_train)

In [79]:
#Test 
from sklearn.linear_model import LinearRegression

reg_test = LinearRegression().fit(X_test, y_test)
y_train_hat = reg_test.predict(X_test)

In [80]:
#Entrainement 
# recherche des meilleurs paramètres
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

parameters = {'alpha': [0.01, 0.1, 1, 10, 100]}

lasso = Lasso()

grid_search = GridSearchCV(lasso, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

BestAlpha = grid_search.best_params_['alpha']

# entraînement du modèle
BestLasso = Lasso(alpha=BestAlpha)
BestLasso.fit(X_train, y_train)



C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.132e+11, tolerance: 2.578e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.145e+11, tolerance: 2.615e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

Lasso(alpha=0.1)

In [81]:
#  meilleur modèle pour prédire le premier dataset
lasso_res = BestLasso.predict(X_train)

# marge d'erreur du modèle
lasso = abs(lasso_res - y_train)


In [82]:
#Test
parameters = {'alpha': [0.01, 0.1, 1, 10, 100]}

lasso = Lasso()

grid_search = GridSearchCV(lasso, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

BestAlpha = grid_search.best_params_['alpha']

# entraînement du modèle
BestLasso_test = Lasso(alpha=BestAlpha)
BestLasso_test.fit(X_train, y_train)

#  meilleur modèle pour prédire le premier dataset
lasso_res = BestLasso_test.predict(X_train)

# marge d'erreur du modèle
lasso = abs(lasso_res - y_train)

C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.132e+11, tolerance: 2.578e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.145e+11, tolerance: 2.615e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

In [87]:
#Entrainement 
from sklearn.ensemble import RandomForestRegressor

# création d'une instance du modèle RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100,max_depth=3, random_state=42)  

# Entraînement du modèle sur les données d'entraînement
rf_reg.fit(X_train, y_train)

#prédictions sur les données d'entraînement
y_train_hat_rf = rf_reg.predict(X_train)




In [88]:
#Test 
# création d'une instance du modèle RandomForestRegressor
rf_reg_test = RandomForestRegressor(n_estimators=100, random_state=42)  

# Entraînement du modèle sur les données d'entraînement
rf_reg_test.fit(X_test, y_test)

#prédictions sur les données d'entraînement
y_train_hat_rf = rf_reg_test.predict(X_test)


In [128]:
#Entrainement 
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Créez une instance du modèle XGBoost pour la régression
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',  # Pour la régression
    learning_rate=1,             # Taux d'apprentissage
    n_estimators=2,              # Nombre d'estimateurs (arbres)
    max_depth=16,                   # Profondeur maximale de l'arbre
    # subsample=0.8,                 # Sous-échantillonnage des lignes
    # colsample_bytree=0.8,          # Sous-échantillonnage des colonnes
    # gamma=0,                       # Paramètre de régularisation
    # reg_lambda=1,                  # Régularisation L2
    # reg_alpha=0,                   # Régularisation L1
    # min_child_weight=1             # Poids minimal des feuilles
)

# Entraînez le modèle sur les données d'entraînement
xgb_reg.fit(X_train, y_train)

# Faites des prédictions sur les données de test
y_test_hat_xgb = xgb_reg.predict(X_train)



In [129]:
#Test 

# Créez une instance du modèle XGBoost pour la régression
xgb_reg_test = xgb.XGBRegressor(
    objective='reg:squarederror',  # Pour la régression
    learning_rate=1,             # Taux d'apprentissage
    n_estimators=2,              # Nombre d'estimateurs (arbres)
    max_depth=16,                   # Profondeur maximale de l'arbre
    # subsample=0.8,                 # Sous-échantillonnage des lignes
    # colsample_bytree=0.8,          # Sous-échantillonnage des colonnes
    # gamma=0,                       # Paramètre de régularisation
    # reg_lambda=1,                  # Régularisation L2
    # reg_alpha=0,                   # Régularisation L1
    # min_child_weight=1             # Poids minimal des feuilles
)

# Entraînez le modèle sur les données d'entraînement
xgb_reg_test.fit(X_test, y_test)

# Faites des prédictions sur les données de test
y_test_hat_xgb = xgb_reg_test.predict(X_test)



## Metrics and Evaluation

In [83]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

def model_evaluator(model_trained):
  temp_y_train_hat = model_trained.predict(X_train)
  print("MAE : ", mean_absolute_error(y_train, temp_y_train_hat))
  print("MSE : ", mean_squared_error(y_train, temp_y_train_hat))
  print("MAPE : ", mean_absolute_percentage_error(y_train, temp_y_train_hat))

In [118]:
list_scores = []

for i_estimator in range(2, 8, 2): 
  for depth_estimator in range(10, 18, 2): 
    bst = xgb.XGBRegressor(n_estimators=i_estimator, max_depth=depth_estimator,
                      learning_rate=1, n_jobs=-1)
    # fit model
    bst.fit(X_train, y_train)
    y_train_hat = bst.predict(X_train)
    y_test_hat = bst.predict(X_test)
    list_scores.append({
        "n_estimator": i_estimator,
        "max_depth": depth_estimator,
        "train_mae":mean_absolute_error(y_train, y_train_hat) ,
        "test_mae": mean_absolute_error(y_test, y_test_hat),
    })

In [119]:

pd.DataFrame(list_scores).sort_values(by="test_mae").head(20)

,n_estimator,max_depth,train_mae,test_mae
5,2,15,49.557979,68.360201
11,3,15,48.060194,68.581148
17,4,15,44.577385,68.755617
23,5,15,41.494598,69.187106
16,4,14,50.501148,69.654047
4,2,14,54.902189,69.754210
29,6,15,39.304985,70.101919
22,5,14,47.057596,70.105083
10,3,14,53.225465,70.366389
35,7,15,37.628948,70.474583


In [113]:
import hyperopt as hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import xgboost as xgb

def score(params):
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid)
    score = mean_absolute_error(y_test, predictions)
    # TODO: Add the importance for the selected features
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    return {'loss': score, 'status': STATUS_OK}

def optimize(
    # trials,
    random_state=4242
):
  space = {
      'n_estimators': hp.quniform('n_estimators', 10, 50, 1),
      # 'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
      # A problem with max_depth casted to float instead of int with
      # the hp.quniform method.
      'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
      # 'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
      # 'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
      # 'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
      # 'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
      'eval_metric': 'mae',
      # 'objective': 'binary:logistic',
      # Increase this number if you have more cores. Otherwise, remove it and it will default
      # to the maxium number.
      'nthread': 4,
      'booster': 'gbtree',
      'tree_method': 'exact',
      'silent': 1,
      'seed': random_state
  }
  best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=150)
  return best

best_hyperparams = optimize(
    # trials
)
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

  0%|          | 0/150 [00:00<?, ?trial/s, best loss=?]

C:\Users\Jawad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [15:11:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[0]	eval-mae:360.90066	train-mae:359.14552             
[1]	eval-mae:350.29000	train-mae:348.85073             
[2]	eval-mae:338.98709	train-mae:337.11634             
  0%|          | 0/150 [00:00<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [84]:
model_evaluator(dummy_regr)
print('\n')
model_evaluator(dummy_regr_test)


MAE :  367.32535901350946
MSE :  814975.2644089637
MAPE :  2.1477637613182426e+17


MAE :  369.32853004749427
MSE :  814986.0320698202
MAPE :  2.1735971654892208e+17


In [85]:
model_evaluator(reg)
print('\n')
model_evaluator(reg_test)

MAE :  363.30870002994044
MSE :  715675.2467702781
MAPE :  1.6423129735370653e+17


MAE :  365.8670507002546
MSE :  715774.6759495926
MAPE :  1.676378698492671e+17


In [86]:
model_evaluator(BestLasso)
print('\n')
model_evaluator(BestLasso_test)

MAE :  363.21141595897046
MSE :  715686.1911543095
MAPE :  1.6413497124993763e+17


MAE :  363.21141595897046
MSE :  715686.1911543095
MAPE :  1.6413497124993763e+17


In [89]:
model_evaluator(rf_reg)
print('\n')
model_evaluator(rf_reg_test)

MAE :  17.379511747779414
MSE :  4213.471042108565
MAPE :  131014254870199.84


MAE :  63.696891892592085
MSE :  50325.2802193782
MAPE :  915758744549116.9


In [130]:
model_evaluator(xgb_reg)
print('\n')
model_evaluator(xgb_reg_test)

MAE :  45.14158192605164
MSE :  21778.0446873478
MAPE :  1884705793687621.8


MAE :  80.81226373250209
MSE :  79858.11197789201
MAPE :  3766627407051585.5
